# Apache Iceberg in Spark

## Demo 1: Apache Iceberg

In [1]:
from pyspark.sql import SparkSession
import os

In [2]:
ICEBERG_REST = "http://iceberg-rest:8181"
MINIO_ENDPOINT = "http://minio:9000"
AWS_REGION = "us-east-1"
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID", "minioadmin")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY", "minioadmin")

In [3]:
print(AWS_ACCESS_KEY_ID)
print(AWS_SECRET_ACCESS_KEY)

minioadmin
minioadmin


In [4]:
spark = (
    SparkSession.builder
    .appName("Iceberg REST via MinIO")
    .master("spark://spark:7077") 
    .config("spark.sql.catalog.ice", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.ice.type", "rest")
    .config("spark.sql.catalog.ice.uri", ICEBERG_REST)
    .config("spark.sql.catalog.ice.warehouse", "s3://warehouse/")
    .config("spark.sql.catalog.ice.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.ice.s3.endpoint", MINIO_ENDPOINT)
    .config("spark.sql.catalog.ice.s3.path-style-access", "true")
    .config("spark.sql.catalog.ice.s3.region", AWS_REGION)
    .config("spark.sql.catalog.ice.s3.access-key-id", AWS_ACCESS_KEY_ID)
    .config("spark.sql.catalog.ice.s3.secret-access-key", AWS_SECRET_ACCESS_KEY)
    .getOrCreate()
)


In [5]:
print("Spark version:", spark.version)

Spark version: 3.5.3


In [6]:
print(spark.sparkContext.master) # should be spark://spark:7077
print(spark.sparkContext.uiWebUrl) # link to the app UI

spark://spark:7077
http://2126d9981963:4040


In [7]:
spark.sql("SHOW NAMESPACES IN ice").show(truncate=False)

+---------+
|namespace|
+---------+
+---------+



In [8]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS ice.demo")

DataFrame[]

In [9]:
spark.sql("SHOW NAMESPACES IN ice").show(truncate=False)

+---------+
|namespace|
+---------+
|demo     |
+---------+



In [10]:
%env AWS_REGION=us-east-1
%env AWS_ACCESS_KEY_ID=minioadmin
%env AWS_SECRET_ACCESS_KEY=minioadmin

env: AWS_REGION=us-east-1
env: AWS_ACCESS_KEY_ID=minioadmin
env: AWS_SECRET_ACCESS_KEY=minioadmin


In [11]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS ice.demo.customers (
        id BIGINT,
        name STRING,
        email STRING
    )
    USING iceberg
    PARTITIONED BY (email)
""")

DataFrame[]

In [12]:
spark.sql("""
    INSERT INTO ice.demo.customers VALUES
      (1, 'Alice Smith', 'alice@example.com'),
      (2, 'Bob Johnson', 'bob@example.com'),
      (3, 'Carol Adams', 'carol@example.com')
""")

DataFrame[]

Select the customers

In [13]:
spark.sql("SELECT * FROM ice.demo.customers").show()

+---+-----------+-----------------+
| id|       name|            email|
+---+-----------+-----------------+
|  3|Carol Adams|carol@example.com|
|  1|Alice Smith|alice@example.com|
|  2|Bob Johnson|  bob@example.com|
+---+-----------+-----------------+



Select the customers with an `o`

In [14]:
spark.sql("SELECT * FROM ice.demo.customers WHERE name like '%o%'").show()

+---+-----------+-----------------+
| id|       name|            email|
+---+-----------+-----------------+
|  3|Carol Adams|carol@example.com|
|  2|Bob Johnson|  bob@example.com|
+---+-----------+-----------------+



Let's add some more data to our DataLake

In [15]:
spark.sql("""
    INSERT INTO ice.demo.customers VALUES
      (4,  'Diego Ramirez',       'diego.ramirez@example.com'),
      (5,  'Maya Patel',          'maya.patel@example.com'),
      (6,  'Liam O’Connor',       'liam.oconnor@example.com'),
      (7,  'Sofia Almeida',       'sofia.almeida@example.com'),
      (8,  'Noah Williams',       'noah.williams@example.com'),
      (9,  'Ava Thompson',        'ava.thompson@example.com'),
      (10, 'Ethan Chen',          'ethan.chen@example.com'),
      (11, 'Olivia Garcia',       'olivia.garcia@example.com'),
      (12, 'Lucas Martin',        'lucas.martin@example.com'),
      (13, 'Emma Robinson',       'emma.robinson@example.com'),
      (14, 'Benjamin Kim',        'benjamin.kim@example.com'),
      (15, 'Isabella Rossi',      'isabella.rossi@example.com'),
      (16, 'James Nguyen',        'james.nguyen@example.com'),
      (17, 'Mila Novak',          'mila.novak@example.com'),
      (18, 'Henry Scott',         'henry.scott@example.com'),
      (19, 'Aria Johnson',        'aria.johnson@example.com'),
      (20, 'Daniela Costa',       'daniela.costa@example.com'),
      (21, 'Jack Wilson',         'jack.wilson@example.com'),
      (22, 'Zoe King',            'zoe.king@example.com'),
      (23, 'Oliver Brown',        'oliver.brown@example.com')
""")

DataFrame[]

Let's run our query

In [16]:
spark.sql("SELECT * FROM ice.demo.customers WHERE name like '%o%'").show()

+---+--------------+--------------------+
| id|          name|               email|
+---+--------------+--------------------+
|  3|   Carol Adams|   carol@example.com|
|  2|   Bob Johnson|     bob@example.com|
| 22|      Zoe King|zoe.king@example.com|
| 21|   Jack Wilson|jack.wilson@examp...|
| 15|Isabella Rossi|isabella.rossi@ex...|
| 18|   Henry Scott|henry.scott@examp...|
|  7| Sofia Almeida|sofia.almeida@exa...|
| 19|  Aria Johnson|aria.johnson@exam...|
| 20| Daniela Costa|daniela.costa@exa...|
|  4| Diego Ramirez|diego.ramirez@exa...|
|  8| Noah Williams|noah.williams@exa...|
| 17|    Mila Novak|mila.novak@exampl...|
|  9|  Ava Thompson|ava.thompson@exam...|
| 13| Emma Robinson|emma.robinson@exa...|
|  6| Liam O’Connor|liam.oconnor@exam...|
| 23|  Oliver Brown|oliver.brown@exam...|
+---+--------------+--------------------+



## Demo 2: Time Travel

Let's view the snapshots, we had two batches, done at different times. So we should see two snapshots

In [17]:
spark.sql("""
  SELECT snapshot_id, committed_at, operation
  FROM ice.demo.customers.snapshots
  ORDER BY committed_at
""").show(truncate=False)

+-------------------+-----------------------+---------+
|snapshot_id        |committed_at           |operation|
+-------------------+-----------------------+---------+
|4519230701306358950|2025-10-09 16:10:35.806|append   |
|8272407367179012364|2025-10-09 16:12:16.888|append   |
+-------------------+-----------------------+---------+



Let's now view the lineage of the data

In [18]:
spark.sql("""
  SELECT made_current_at, snapshot_id, parent_id, is_current_ancestor
  FROM ice.demo.customers.history
  ORDER BY made_current_at
""").show(truncate=False)

+-----------------------+-------------------+-------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id          |is_current_ancestor|
+-----------------------+-------------------+-------------------+-------------------+
|2025-10-09 16:10:35.806|4519230701306358950|NULL               |true               |
|2025-10-09 16:12:16.888|8272407367179012364|4519230701306358950|true               |
+-----------------------+-------------------+-------------------+-------------------+



Now, let's go back in time by reverting the timestamp

In [22]:
spark.conf.set("spark.sql.session.timeZone", "UTC")

# Replace the following timestamp with one after the initial timestamp
ts = "2025-10-09 16:11:00.000"

df_ts = spark.sql(f"""
  SELECT *
  FROM ice.demo.customers
  TIMESTAMP AS OF '{ts}'
""")
df_ts.show()

+---+-----------+-----------------+
| id|       name|            email|
+---+-----------+-----------------+
|  3|Carol Adams|carol@example.com|
|  1|Alice Smith|alice@example.com|
|  2|Bob Johnson|  bob@example.com|
+---+-----------+-----------------+



In [23]:
spark.sql("""SELECT snapshot_id, committed_at, operation
FROM ice.demo.customers.snapshots
ORDER BY committed_at DESC
LIMIT 5;""").show(truncate=False)

+-------------------+-----------------------+---------+
|snapshot_id        |committed_at           |operation|
+-------------------+-----------------------+---------+
|8272407367179012364|2025-10-09 16:12:16.888|append   |
|4519230701306358950|2025-10-09 16:10:35.806|append   |
+-------------------+-----------------------+---------+



## Demo 3: Changing the Schema

In [24]:
spark.sql("""DESCRIBE TABLE ice.demo.customers;""").show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|                  id|   bigint|   NULL|
|                name|   string|   NULL|
|               email|   string|   NULL|
|# Partition Infor...|         |       |
|          # col_name|data_type|comment|
|               email|   string|   NULL|
+--------------------+---------+-------+



In [25]:
spark.sql("""ALTER TABLE ice.demo.customers
ADD COLUMN country STRING COMMENT 'ISO 3166-1 code' AFTER email;""")

DataFrame[]

In [26]:
spark.sql("""DESCRIBE TABLE ice.demo.customers;""").show()

+--------------------+---------+---------------+
|            col_name|data_type|        comment|
+--------------------+---------+---------------+
|                  id|   bigint|           NULL|
|                name|   string|           NULL|
|               email|   string|           NULL|
|             country|   string|ISO 3166-1 code|
|# Partition Infor...|         |               |
|          # col_name|data_type|        comment|
|               email|   string|           NULL|
+--------------------+---------+---------------+



In [27]:
spark.sql("""UPDATE ice.demo.customers
SET country = 'US'
WHERE email LIKE '%@example.com';""").show()

++
||
++
++



In [28]:
spark.sql("SELECT * FROM ice.demo.customers").show()

+---+--------------+--------------------+-------+
| id|          name|               email|country|
+---+--------------+--------------------+-------+
| 22|      Zoe King|zoe.king@example.com|     US|
| 21|   Jack Wilson|jack.wilson@examp...|     US|
| 15|Isabella Rossi|isabella.rossi@ex...|     US|
|  1|   Alice Smith|   alice@example.com|     US|
|  2|   Bob Johnson|     bob@example.com|     US|
| 12|  Lucas Martin|lucas.martin@exam...|     US|
| 18|   Henry Scott|henry.scott@examp...|     US|
| 16|  James Nguyen|james.nguyen@exam...|     US|
|  5|    Maya Patel|maya.patel@exampl...|     US|
|  7| Sofia Almeida|sofia.almeida@exa...|     US|
| 19|  Aria Johnson|aria.johnson@exam...|     US|
|  3|   Carol Adams|   carol@example.com|     US|
| 20| Daniela Costa|daniela.costa@exa...|     US|
|  4| Diego Ramirez|diego.ramirez@exa...|     US|
| 17|    Mila Novak|mila.novak@exampl...|     US|
|  8| Noah Williams|noah.williams@exa...|     US|
|  9|  Ava Thompson|ava.thompson@exam...|     US|


Let's go back before we edited the schema. Here is a list of operations in Iceberg

| Operation            | Meaning                                                                 | Typical Trigger / Example                                     |
|----------------------|-------------------------------------------------------------------------|---------------------------------------------------------------|
| **append**           | Adds new data files to the table without touching existing ones         | `INSERT INTO ...`, batch ingest                               |
| **overwrite**        | Replaces existing data files with new ones                              | `INSERT OVERWRITE`, Spark `.mode("overwrite")` writes         |
| **replace partitions** | Overwrites only affected partitions, leaving others intact            | Dynamic partition overwrite in Spark streaming                |
| **delete**           | Removes rows from files (position deletes or equality deletes)          | `DELETE FROM table WHERE ...`                                 |
| **update**           | Updates rows (internally: delete + insert of modified rows)             | `UPDATE table SET ... WHERE ...`                              |
| **rewrite** (or `replace`) | Rewrites data files without logical changes (optimization/compaction) | `REWRITE DATA`, clustering, file compaction                   |
| **fast-append**      | Fast ingestion, skips some validation checks (legacy mode)              | Optimized append from some engines                            |

In [31]:
spark.sql("""SELECT snapshot_id, committed_at, operation
FROM ice.demo.customers.snapshots
ORDER BY committed_at DESC
LIMIT 5;""").show(truncate=False)

+-------------------+-----------------------+---------+
|snapshot_id        |committed_at           |operation|
+-------------------+-----------------------+---------+
|5989591670914269974|2025-10-09 16:13:53.304|overwrite|
|8272407367179012364|2025-10-09 16:12:16.888|append   |
|4519230701306358950|2025-10-09 16:10:35.806|append   |
+-------------------+-----------------------+---------+



Notice at this point that there is the old schema since we had a different schema back then

In [32]:
spark.sql("""SELECT *
FROM ice.demo.customers
VERSION AS OF '8272407367179012364';""").show(truncate=False)

+---+--------------+--------------------------+
|id |name          |email                     |
+---+--------------+--------------------------+
|22 |Zoe King      |zoe.king@example.com      |
|21 |Jack Wilson   |jack.wilson@example.com   |
|15 |Isabella Rossi|isabella.rossi@example.com|
|12 |Lucas Martin  |lucas.martin@example.com  |
|18 |Henry Scott   |henry.scott@example.com   |
|16 |James Nguyen  |james.nguyen@example.com  |
|5  |Maya Patel    |maya.patel@example.com    |
|7  |Sofia Almeida |sofia.almeida@example.com |
|19 |Aria Johnson  |aria.johnson@example.com  |
|20 |Daniela Costa |daniela.costa@example.com |
|4  |Diego Ramirez |diego.ramirez@example.com |
|8  |Noah Williams |noah.williams@example.com |
|17 |Mila Novak    |mila.novak@example.com    |
|9  |Ava Thompson  |ava.thompson@example.com  |
|11 |Olivia Garcia |olivia.garcia@example.com |
|13 |Emma Robinson |emma.robinson@example.com |
|10 |Ethan Chen    |ethan.chen@example.com    |
|6  |Liam O’Connor |liam.oconnor@example

## Demo 4: Deletes

In [33]:
spark.sql("""DELETE FROM ice.demo.customers WHERE name like '%ak%'""")

DataFrame[]

Next let's show the results

In [34]:
spark.sql("""SELECT *
FROM ice.demo.customers;""").show(truncate=False)

+---+--------------+--------------------------+-------+
|id |name          |email                     |country|
+---+--------------+--------------------------+-------+
|22 |Zoe King      |zoe.king@example.com      |US     |
|21 |Jack Wilson   |jack.wilson@example.com   |US     |
|15 |Isabella Rossi|isabella.rossi@example.com|US     |
|1  |Alice Smith   |alice@example.com         |US     |
|2  |Bob Johnson   |bob@example.com           |US     |
|12 |Lucas Martin  |lucas.martin@example.com  |US     |
|18 |Henry Scott   |henry.scott@example.com   |US     |
|16 |James Nguyen  |james.nguyen@example.com  |US     |
|5  |Maya Patel    |maya.patel@example.com    |US     |
|7  |Sofia Almeida |sofia.almeida@example.com |US     |
|19 |Aria Johnson  |aria.johnson@example.com  |US     |
|3  |Carol Adams   |carol@example.com         |US     |
|20 |Daniela Costa |daniela.costa@example.com |US     |
|4  |Diego Ramirez |diego.ramirez@example.com |US     |
|8  |Noah Williams |noah.williams@example.com |U

Let's view the snapshots that have been committed up to this point and we will notice a `delete` operation as part of the snapshot history

In [36]:
spark.sql("""SELECT snapshot_id, committed_at, operation
FROM ice.demo.customers.snapshots
ORDER BY committed_at DESC
LIMIT 5;""").show(truncate=False)

+-------------------+-----------------------+---------+
|snapshot_id        |committed_at           |operation|
+-------------------+-----------------------+---------+
|516322263074055114 |2025-10-09 16:14:31.448|delete   |
|5989591670914269974|2025-10-09 16:13:53.304|overwrite|
|8272407367179012364|2025-10-09 16:12:16.888|append   |
|4519230701306358950|2025-10-09 16:10:35.806|append   |
+-------------------+-----------------------+---------+



Again, let's go back in time, and see if we can see Mila

In [37]:
spark.sql("""SELECT *
FROM ice.demo.customers
VERSION AS OF '5989591670914269974';""").show(truncate=False)

+---+--------------+--------------------------+-------+
|id |name          |email                     |country|
+---+--------------+--------------------------+-------+
|22 |Zoe King      |zoe.king@example.com      |US     |
|21 |Jack Wilson   |jack.wilson@example.com   |US     |
|15 |Isabella Rossi|isabella.rossi@example.com|US     |
|1  |Alice Smith   |alice@example.com         |US     |
|2  |Bob Johnson   |bob@example.com           |US     |
|12 |Lucas Martin  |lucas.martin@example.com  |US     |
|18 |Henry Scott   |henry.scott@example.com   |US     |
|16 |James Nguyen  |james.nguyen@example.com  |US     |
|5  |Maya Patel    |maya.patel@example.com    |US     |
|7  |Sofia Almeida |sofia.almeida@example.com |US     |
|19 |Aria Johnson  |aria.johnson@example.com  |US     |
|3  |Carol Adams   |carol@example.com         |US     |
|20 |Daniela Costa |daniela.costa@example.com |US     |
|4  |Diego Ramirez |diego.ramirez@example.com |US     |
|17 |Mila Novak    |mila.novak@example.com    |U

In [38]:
spark.stop()